In [1]:
import os
from tqdm import tqdm

import pandas as pd
from parse_tool import PDFProcessor
from text_processing import *


In [2]:
# NAMESPACE = 'SANTAFE_MX5_2023'
NAMESPACE = 'TESLA_MODEL3'
# pdf_file = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_parser/pdf/2024 아이오닉 5 (IONIQ 5) NE.pdf'

# pdf_file = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_parser/pdf/디 올 뉴 싼타페 MX5.pdf'
pdf_file = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_parser/pdf/TESLA_MODEL3.pdf'
context_file_root_path = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_context'

os.makedirs(context_file_root_path + f'/image/{NAMESPACE}', exist_ok=True)
os.makedirs(context_file_root_path + f'/csv/{NAMESPACE}', exist_ok=True)
os.makedirs(context_file_root_path + f'/text/{NAMESPACE}', exist_ok=True)
context_text_dir = context_file_root_path + f'/text/{NAMESPACE}'

fitz_page_start = 7
fitz_page_end = 253
HYUNNDAI_y_range = (39, 575)
TESLA_y_range=(39,775)

In [3]:
pdf_processor = PDFProcessor(NAMESPACE, pdf_file, fitz_page_start, fitz_page_end, y_range=TESLA_y_range, context_file_root_path = context_file_root_path)

In [4]:
span_list = []
contents_list = []
img_urls = []
csv_urls = []
contents_type = []
block_nums = []
while 1:
    try:
        if pdf_processor.page_num % 50 == 0:
            print(f"Processing page {pdf_processor.page_num}")
        next(pdf_processor)
        page_result = pdf_processor.parse_pdf()
        
        span_list.extend(page_result['span'])
        contents_list.extend(page_result['contents'])
        csv_urls.extend(page_result['csv_urls'])
        img_urls.extend(page_result['img_urls'])
        contents_type.extend(page_result['contents_type'])
        block_nums.extend(page_result['block_num'])
    except StopIteration as e:
        print('End of processing', e)
        break
    except Exception as e:
        print(f"Error raised: {e}, page_num: {pdf_processor.page_num}")
        break

Processing page 50
Processing page 100
Processing page 150
Processing page 200
Processing page 250
End of processing 


In [5]:
df = pd.DataFrame({
    'span': span_list,
    'contents': contents_list,
    'csv_urls': csv_urls,
    'img_urls': img_urls,
    'ctype': contents_type,
    'block_num': block_nums
})

df = df[(df['ctype']=='image') | (~df['contents'].isna())]

In [6]:
df.to_csv(f'{context_text_dir}/{NAMESPACE}_result.csv', index=False)
df.to_parquet(f'{context_text_dir}/{NAMESPACE}_result.parquet', index=False)

In [7]:
df = pd.read_parquet(f'{context_text_dir}/{NAMESPACE}_result.parquet')

In [8]:
df

,span,contents,csv_urls,img_urls,ctype,block_num
0,"""('NotoSansKR-Regular', 0, 9)"", ""('NotoSansKR-...",참고: 이 사용자 매뉴얼에서 전면 터치스크린은 “터치스크린”이라고 하고 후면 터치스...,None,None,text,8_0
1,"""('NotoSansKR-Regular', 0, 9)"", ""('NotoSansKR-...",경고: 운전 시 도로 및 교통 상황에 항상 주의를 기울이십시오. 운전자 주의 산만을...,None,None,text,8_1
2,"""('NotoSansKR-Regular', 0, 9)""",을 보장하려면 차량이 움직이는 동안 터치스크린을 사용하여 설정을 조정하지 마십시오.,None,None,text,8_1
3,None,None,None,image/TESLA_MODEL3/7_2_img.png,image,8_2
4,"""('NotoSansKR-Regular', 0, 9)""",터치스크린은 일반적인 차량에서 물리적 버튼을 사용하여 제어하는 많은 기능(예: 실내...,None,None,text,8_3
...,...,...,...,...,...,...
8674,"""('NotoSansKR-Regular', 0, 9)""","테스트 절차는 제조 중, 그리고 제조 이후에 진행된 폭넓은 검사를 포",None,None,text,253_4
8675,"""('NotoSansKR-Regular', 0, 9)""","함합니다. 최종 검사는 Tesla에서 이루어지며, 기술자가 수행하는 도",None,None,text,253_4
8676,"""('NotoSansKR-Regular', 0, 9)""",로 테스트를 포함합니다.,None,None,text,253_4
8677,"""('NotoSansKR-Bold', 0, 14)""",사운드 라이브러리,None,None,text,253_5


In [21]:
df = remove_invalid_row_by_string(df)
df = remove_invalid_row_by_span(df)
df = add_header(df)

In [22]:
block_df = group_by_block(df)
doc_df = group_by_doc(block_df)

In [23]:
doc_df.to_parquet(f'{context_text_dir}/doc_{NAMESPACE}_result.parquet')